In [9]:
import os
import dotenv
from pprint import pprint

dotenv.load_dotenv()
GRID_API_KEY = os.getenv("GRID_API_KEY")
LOL_TITLE_ID = 3
STATIC_AAPI_URL = "https://api-op.grid.gg/central-data/graphql/"


In [ ]:
import requests
def fetch_from_grid(
        query:str,
        variables:str = None,
        url:str = "https://api-op.grid.gg/central-data/graphql/",
        api_key:str = GRID_API_KEY,
) -> requests.Response:
    headers = {
        "x-api-key": api_key
    }

    req_json = {
        "query": query
    }

    if variables:
        req_json['variables'] = variables

    res = requests.post(
        url,
        headers=headers,
        json = req_json
    )

    return res.json()

# grab schema


In [37]:
import json

body = """
{
  __schema {
    queryType {
      fields{
        name
        description
        args {
          name
          description
          type {
            name
            description
            kind
            ofType {
              name
              kind
            }
          }
        }
        type {
          name
          description
          kind
          ofType {
            name
            kind
          }
        }
      }
    }
  }
}
"""

res = fetch_from_grid(body, STATIC_AAPI_URL, GRID_API_KEY)

with open("central-data-query-schema.json", 'w') as f:
    json.dump(res, f, indent=2)

In [ ]:

body = """
{
    __schema {
        types {
            name
            description
            kind
            fields {
                name
                description
                type {
                    name
                    description
                    kind
                    ofType {
                        name
                        kind
                    }
                }
                args {
                    name
                    description
                    type{
                        name
                        kind
                        ofType {
                            name
                            kind
                        }
                    }
                }
            }
        }
    }
}
"""

res = fetch_from_grid(body, STATIC_AAPI_URL, GRID_API_KEY)

with open("central-data-types-schema.json", 'w') as f:
    json.dump(res, f, indent=2)

# STATIC DATA TEST

In [ ]:
#directly use f-string, a bit annoying because of formatting, but don't need to name the query
body = f"""
{{
    allSeries (
        filter: {{
            titleId: {LOL_TITLE_ID}
        }}
    ) {{
        totalCount,
        pageInfo {{
          hasPreviousPage
          hasNextPage
          startCursor
          endCursor
        }}
        edges {{
          cursor
          node {{
              id
              title {{
                nameShortened
              }}
              tournament {{
                nameShortened
              }}
              startTimeScheduled
              format {{
                name
                nameShortened
              }}
              teams {{
                baseInfo {{
                  name
                }}
                scoreAdvantage
              }}
          }}
        }}
      }}
}}
"""
res = fetch_from_grid(body)
pprint(res)

{'data': {'allSeries': {'edges': [{'cursor': 'PjMLC1oLC0VBRkFEQEYLC1oLCw==',
                                   'node': {'format': {'name': 'best-of-3',
                                                       'nameShortened': 'Bo3'},
                                            'id': '2616371',
                                            'startTimeScheduled': '2024-01-17T08:00:00Z',
                                            'teams': [{'baseInfo': {'name': 'NONGSHIM '
                                                                            'RED '
                                                                            'FORCE'},
                                                       'scoreAdvantage': 0},
                                                      {'baseInfo': {'name': 'DRX'},
                                                       'scoreAdvantage': 0}],
                                            'title': {'nameShortened': 'lol'},
                                         

In [ ]:
#query using graphql variables instead, more flexible/cleaner ig
query = """
query GetSeriesByTitle ($gameId: ID) {
    allSeries (
        filter: {
            titleId: $gameId
        }
    ) {
        totalCount,
        pageInfo {
          hasPreviousPage
          hasNextPage
          startCursor
          endCursor
        }
        edges {
          cursor
          node {
              id
              title {
                nameShortened
              }
              tournament {
                nameShortened
              }
              startTimeScheduled
              format {
                name
                nameShortened
              }
              teams {
                baseInfo {
                  name
                }
                scoreAdvantage
              }
              players {
                id
                fullName
                nickname
                age
                roles {
                  name
                }
                team {
                  id
                  name
                  nameShortened
                  organization {
                    name
                  }
                  rating
                  updatedAt
                }
                updatedAt
              }
          }
        }
      }
}
"""

variables = {
    "gameId": LOL_TITLE_ID
}

res = fetch_from_grid(query, variables=variables)
pprint(res)

{'data': {'allSeries': {'edges': [{'cursor': 'PjMLC1oLC0VBRkFEQEYLC1oLCw==',
                                   'node': {'format': {'name': 'best-of-3',
                                                       'nameShortened': 'Bo3'},
                                            'id': '2616371',
                                            'players': [],
                                            'startTimeScheduled': '2024-01-17T08:00:00Z',
                                            'teams': [{'baseInfo': {'name': 'NONGSHIM '
                                                                            'RED '
                                                                            'FORCE'},
                                                       'scoreAdvantage': 0},
                                                      {'baseInfo': {'name': 'DRX'},
                                                       'scoreAdvantage': 0}],
                                            'title': {'nameSh

In [ ]:
#tournament testing
query = """
query GetTournamentsByTitle ($gameId: ID) {
    tournaments (
        filter: {
            titleId: $gameId
        }
    ) {
        totalCount,
        pageInfo {
          hasPreviousPage
          hasNextPage
          startCursor
          endCursor
        }
        edges {
          cursor
          node {
              id
              name
              parent {
                id
                name
              }
              teams {
                id
                name
              }

          }
        }
      }
}
"""

variables = {
    "gameId": LOL_TITLE_ID
}

res = fetch_from_grid(query, variables=variables)
pprint(res)

{'data': {'tournaments': {'edges': [{'cursor': 'PjMLC1oLC0BCQU5HQAsLWgsL',
                                     'node': {'id': '756907',
                                              'name': 'LCK - Spring 2024 '
                                                      '(Regular Season: '
                                                      'Regular Season)',
                                              'parent': {'id': '758025',
                                                         'name': 'LCK - Spring '
                                                                 '2024 '
                                                                 '(Regular '
                                                                 'Season)'},
                                              'teams': []}},
                                    {'cursor': 'PjMLC1oLC0BCQU5HTwsLWgsL',
                                     'node': {'id': '756908',
                                              'name': 'LCK - S

# SERIES IN-GAME DATA TEST (this is for live data)


In [ ]:
AAPI_URL = "https://api-op.grid.gg/live-data-feed/series-state/graphql"


